Date Created - 6th November 2020

Date Updates - 6th November 2020

Created by - Akshay Chougule

## Basics

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import json
# from pandas.io.json import json_normalize
# import xlrd

In [2]:
# import plotly.express as px 
# import plotly.graph_objects as go

import dash  
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table_experiments as dt

In [3]:
import dash_table

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
nctid = 'NCT01859988'

## AE Fetching Functions

In [6]:
def get_data(query_url):
    r = requests.get(query_url)
    j = json.loads(r.content)
    return(pd.DataFrame(j['results']))

def get_ae_pt_data(drug_brand_name):
    male_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    female_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:1+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    male_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:2+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    female_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:2+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    df_male_ser = get_data(male_serious_url)
    df_male_ser['gender']='male'
    df_male_ser['serious']=1
    df_female_ser = get_data(female_serious_url)
    df_female_ser['gender']='female'
    df_female_ser['serious']=1
    df_male_nonser = get_data(male_nonserious_url)
    df_male_nonser['gender']='male'
    df_male_nonser['serious']=0
    df_female_nonser = get_data(female_nonserious_url)
    df_female_nonser['gender']='female'
    df_female_nonser['serious']=0
    return(pd.concat([df_male_ser, df_female_ser,df_male_nonser,df_female_nonser], ignore_index=True))

## Testing

In [7]:
tt = get_ae_pt_data('avastin')
tt.head()

,term,count,gender,serious
0,DEATH,1544,male,1
1,DIARRHOEA,839,male,1
2,OFF LABEL USE,791,male,1
3,DISEASE PROGRESSION,716,male,1
4,FATIGUE,613,male,1


In [8]:
tt = get_ae_pt_data('humira')
tt.head()

,term,count,gender,serious
0,DRUG INEFFECTIVE,4025,male,1
1,CROHN^S DISEASE,2730,male,1
2,ARTHRALGIA,2308,male,1
3,PAIN,2104,male,1
4,DEATH,1738,male,1


## Dash App

In [9]:
app = dash.Dash(__name__)

In [ ]:
# App layout
app.layout = html.Div([

    html.H1("Safety Data Explorer", style={'text-align': 'center'}),

    html.Div([
        html.Br(),
        dcc.Input(id="trial1", type="text", placeholder="Enter Drug (brand) Name", debounce =True),
        html.Br(),
    ]),
    html.Br(),   
    html.Br(), html.Br(),
    
    dcc.Tabs([
        dcc.Tab(label='Explore FAERS Data', children=[
            html.Div(id='blank4', style={'width': '7%', 'display': 'inline-block'}),    
            html.Div(id='output-sae-subs', style={'width': '85%', 'display': 'inline-block'}),
        ]),
     
        dcc.Tab(label='Explore Trial Safety Data', children=[
            html.Div(id='blank5', style={'width': '7%', 'display': 'inline-block'}),    
            html.Div(id='output-oae-subs', style={'width': '85%', 'display': 'inline-block'}),
        ]),
        dcc.Tab(label='Compare', children=[
            
        ])
     ]),
    
    html.Br(),
])


# ------------------------------------------------------------------------------
# Connect the data with Dash Components
@app.callback(
    [dash.dependencies.Output('output-sae-subs', 'children'),
     dash.dependencies.Output('output-oae-subs', 'children')],
    Input(component_id='trial1', component_property='value')
)


# update logic
def update_graph(drug_brand_name):
    print(drug_brand_name)  
    
    df = get_ae_pt_data(drug_brand_name)

    sae_tab = dash_table.DataTable(
                    id='table',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict("rows"),
                    filter_action="native",
                    export_format="csv",
                    export_headers="display",
                    style_cell={'width': '300px',
                    'height': '60px',
                    'textAlign': 'left'},
                    style_data_conditional=[{
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }],
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    })
       
    oae_tab=dash_table.DataTable(
                id='table2',
                columns=[{"name": i, "id": i} for i in df.columns],
                data=df.to_dict("rows"),
                filter_action="native",
                export_format="csv",
                export_headers="display",
                style_cell={'width': '300px',
                'height': '60px',
                'textAlign': 'left'},
                style_data_conditional=[{
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }],
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                })


    return html.Div([
        html.H4(' ', style={'text-align': 'center'}),
        sae_tab
    ]), html.Div([
        html.H4(' ', style={'text-align': 'center'}),
        oae_tab
    ])


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Nov/2020 10:02:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2020 10:02:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2020 10:02:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2020 10:02:42] "GET /_favicon.ico?v=1.16.0 HTTP/1.1" 200 -


None


/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

127.0.0.1 - - [07/Nov/2020 10:02:43] "POST /_dash-update-component HTTP/1.1" 200 -
